In [19]:
# Import our dependencies
from statistics import mean
from sklearn.model_selection import train_test_split, cross_validate, RepeatedStratifiedKFold
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
import pandas as pd
import tensorflow as tf
from imblearn.pipeline import Pipeline

In [2]:
# Import our input dataset
df = pd.read_csv('../eda/reduced_filtered_df.csv')
df.head()

,ACD,LT,VCD,SPORTHR,DADMY,delta_spheq,total_positive_screen,MYOPIC
0,3.702,3.392,15.29,4,1,1.358,8,0
1,3.462,3.514,15.52,14,0,1.929,10,0
2,3.224,3.556,15.36,10,1,2.494,26,0
3,3.186,3.654,15.49,12,1,1.433,16,0
4,3.732,3.584,15.08,12,0,2.022,8,0


In [3]:
# Define X,y
label = df["MYOPIC"].copy()
X = df.iloc[:,:-1].copy()
X.head()

# Split training/test datasets
X_train, X_test, y_train, y_test = train_test_split(X, label, random_state=42, stratify=label)

In [4]:
# Preprocess numerical data for neural network

# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [5]:
# Define the deep learning model 
nn_model = tf.keras.models.Sequential()
nn_model.add(tf.keras.layers.Dense(units=16, activation="relu", input_dim=7))
nn_model.add(tf.keras.layers.Dense(units=16, activation="relu"))
nn_model.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Compile the Sequential model together and customize metrics
nn_model.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

# Train the model
fit_model = nn_model.fit(X_train_scaled, y_train, epochs=50)

# Evaluate the model using the test data
model_loss, model_accuracy = nn_model.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

Epoch 1/50
9/9 [==============================] - 0s 665us/step - loss: 0.7966 - accuracy: 0.3405
Epoch 2/50
9/9 [==============================] - 0s 1ms/step - loss: 0.7165 - accuracy: 0.4839
Epoch 3/50
9/9 [==============================] - 0s 554us/step - loss: 0.6522 - accuracy: 0.6093
Epoch 4/50
9/9 [==============================] - 0s 554us/step - loss: 0.6032 - accuracy: 0.7276
Epoch 5/50
9/9 [==============================] - 0s 554us/step - loss: 0.5613 - accuracy: 0.8387
Epoch 6/50
9/9 [==============================] - 0s 886us/step - loss: 0.5298 - accuracy: 0.8602
Epoch 7/50
9/9 [==============================] - 0s 658us/step - loss: 0.5021 - accuracy: 0.8638
Epoch 8/50
9/9 [==============================] - 0s 444us/step - loss: 0.4790 - accuracy: 0.8674
Epoch 9/50
9/9 [==============================] - 0s 554us/step - loss: 0.4573 - accuracy: 0.8674
Epoch 10/50
9/9 [==============================] - 0s 1ms/step - loss: 0.4370 - accuracy: 0.8674
Epoch 11/50
9/9 [======

# Predict results and evaluate

In [6]:
y_pred = nn_model.predict(X_test_scaled)

In [7]:
# convert to binary output
y_pred_binary = [0 if i <=0.5 else 1 for i in y_pred]


In [8]:
confusion_matrix(y_test, y_pred_binary)

array([[81,  0],
       [10,  2]], dtype=int64)

In [9]:
print(classification_report(y_test, y_pred_binary))

              precision    recall  f1-score   support

           0       0.89      1.00      0.94        81
           1       1.00      0.17      0.29        12

    accuracy                           0.89        93
   macro avg       0.95      0.58      0.61        93
weighted avg       0.90      0.89      0.86        93



# Changing the Threshold Criteria

In [10]:
# convert to binary output
y_pred_binary = [0 if i <=0.3 else 1 for i in y_pred]

In [11]:
confusion_matrix(y_test, y_pred_binary)

array([[78,  3],
       [ 8,  4]], dtype=int64)

In [12]:
print(classification_report(y_test, y_pred_binary))

              precision    recall  f1-score   support

           0       0.91      0.96      0.93        81
           1       0.57      0.33      0.42        12

    accuracy                           0.88        93
   macro avg       0.74      0.65      0.68        93
weighted avg       0.86      0.88      0.87        93



## Pipeline for Neural Network

In [17]:
# create process steps
pipes = [
    ("scaler", StandardScaler()),
    ("nn", nn_model)
]

# create pipeline
pipeline = Pipeline(pipes)

# Train the scaler with the X_train data.
pipeline.fit(X_train, y_train)

# # evaluate model
# print(f"Score:  {pipeline.score(X_train, y_train):>10}")

y_train_pred = pipeline.predict(X_train)
y_pred_binary = [0 if i <=0.3 else 1 for i in y_train_pred]
print(f" Training Classification Report:  \n {classification_report(y_train, y_pred_binary)}")
print(f"-----"*10)
y_test_pred = pipeline.predict(X_test)
y_pred_binary = [0 if i <=0.3 else 1 for i in y_test_pred]
print(f" Testing Classification Report:  \n {classification_report(y_test, y_pred_binary)}")

9/9 [==============================] - 0s 1ms/step - loss: 0.2462 - accuracy: 0.8961
 Training Classification Report:  
               precision    recall  f1-score   support

           0       0.93      0.92      0.93       242
           1       0.53      0.57      0.55        37

    accuracy                           0.87       279
   macro avg       0.73      0.74      0.74       279
weighted avg       0.88      0.87      0.88       279

--------------------------------------------------
 Testing Classification Report:  
               precision    recall  f1-score   support

           0       0.91      0.96      0.93        81
           1       0.57      0.33      0.42        12

    accuracy                           0.88        93
   macro avg       0.74      0.65      0.68        93
weighted avg       0.86      0.88      0.87        93



## Cross Validation for Neural Network

In [21]:
# # create process steps
# pipes = [
#     ("scaler", StandardScaler()),
#     ("nn", nn_model)
# ]

# def create_model():
#     # Define the deep learning model 
#     nn_model = tf.keras.models.Sequential()
#     nn_model.add(tf.keras.layers.Dense(units=16, activation="relu", input_dim=7))
#     nn_model.add(tf.keras.layers.Dense(units=16, activation="relu"))
#     nn_model.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))
#     return nn_module

# # Compile the Sequential model together and customize metrics
# nn_model.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

# # create pipeline
# pipeline = Pipeline(pipes)

# # cross validation
# cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=1)

# scoring = ('f1', 'recall', 'precision', 'roc_auc')

# model = KerasClassifier(model=create_model, epochs=150, batch_size=10, verbose=0)

# scores = cross_validate(model, X_train, y_train, scoring=scoring, cv=cv, n_jobs=-1)
# print(f"-----"*10)
# print("\n")
# print(f"Cross Validation Prediction Scores - average of 30 runs")
# print('Mean f1: %.3f' % mean(scores['test_f1']))
# print('Mean recall: %.3f' % mean(scores['test_recall']))
# print('Mean precision: %.3f' % mean(scores['test_precision']))
# print('Mean ROC AUC: %.3f' % mean(scores['test_roc_auc']))
# print("\n")